In [1]:
import cv2
import time
import mediapipe as mp

In [5]:
def storeKeyPoints(results):

    keyPoints = {}
    for idx, lm in enumerate(results.pose_landmarks.landmark):
        keyPoints[idx] = { "x" : lm.x,
                           "y" : lm.y,
                           "z" : lm.z,
                           "v" : lm.visibility}


    keyPoints[33] = {"x" : (keyPoints[24]["x"] + keyPoints[23]["x"]) / 2,
                     "y" : (keyPoints[24]["y"] + keyPoints[23]["y"]) / 2}
    
    return keyPoints


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose


# For webcam input:
cap = cv2.VideoCapture("/Users/calvin/Downloads/tv_human_interactions_videos/handShake_0029.avi")
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
    pose.enable_segmentation = False
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring emtpty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            break
            #continue
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        # Flip the image horizontally for a selfie-view display.
        #image = cv2.circle(image, (int(keyPoints[33]["x"]),int(keyPoints[33]["y"])), radius=0, color=(0, 0, 255), thickness=-1)
        cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
        if cv2.waitKey(100) & (0xFF == 27 | 0xFF == ord("q")):
            break
    cap.release()

Ignoring emtpty camera frame.


In [84]:
import math

In [86]:


#math.sqrt(((hip_left["x"] - hip_right["x"]) ** 2) + ((hip_left["y"] - hip_right["y"]) ** 2))



In [87]:
keyPoints

{0: {'x': 0.6008322238922119,
  'y': 0.462470680475235,
  'z': -1.1670618057250977,
  'v': 0.9998276233673096},
 1: {'x': 0.633931040763855,
  'y': 0.3587099313735962,
  'z': -1.098837971687317,
  'v': 0.9997501969337463},
 2: {'x': 0.6548106670379639,
  'y': 0.3525341749191284,
  'z': -1.099186658859253,
  'v': 0.9997151494026184},
 3: {'x': 0.6739158034324646,
  'y': 0.34686279296875,
  'z': -1.100128173828125,
  'v': 0.9997707009315491},
 4: {'x': 0.5565930008888245,
  'y': 0.37000012397766113,
  'z': -1.1022462844848633,
  'v': 0.9996305704116821},
 5: {'x': 0.5322360992431641,
  'y': 0.3685295581817627,
  'z': -1.101444125175476,
  'v': 0.999515175819397},
 6: {'x': 0.5124530792236328,
  'y': 0.36798450350761414,
  'z': -1.1017333269119263,
  'v': 0.9995390176773071},
 7: {'x': 0.7035438418388367,
  'y': 0.38108837604522705,
  'z': -0.5874524116516113,
  'v': 0.9998277425765991},
 8: {'x': 0.4913698136806488,
  'y': 0.4020073413848877,
  'z': -0.5731522440910339,
  'v': 0.99955266

In [ ]:
NOSE = 0
LEFT_EYE_INNER = 1
LEFT_EYE = 2
LEFT_EYE_OUTER = 3
RIGHT_EYE_INNER = 4
RIGHT_EYE = 5
RIGHT_EYE_OUTER = 6
LEFT_EAR = 7
RIGHT_EAR = 8
MOUTH_LEFT = 9
MOUTH_RIGHT = 10
LEFT_SHOULDER = 11
RIGHT_SHOULDER = 12
LEFT_ELBOW = 13
RIGHT_ELBOW = 14
LEFT_WRIST = 15
RIGHT_WRIST = 16
LEFT_PINKY = 17
RIGHT_PINKY = 18
LEFT_INDEX = 19
RIGHT_INDEX = 20
LEFT_THUMB = 21
RIGHT_THUMB = 22
LEFT_HIP = 23
RIGHT_HIP = 24
LEFT_KNEE = 25
RIGHT_KNEE = 26
LEFT_ANKLE = 27
RIGHT_ANKLE = 28
LEFT_HEEL = 29
RIGHT_HEEL = 30
LEFT_FOOT_INDEX = 31
RIGHT_FOOT_INDEX = 32


In [ ]:
# For static images:
IMAGE_FILES = []
BG_COLOR = (192, 192, 192) # gray
with mp_pose.Pose(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5) as pose:
    for idx, file in enumerate(IMAGE_FILES):
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        # Convert the BGR image to RGB before processing.
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if not results.pose_landmarks:
            continue
        print(
            f'Nose coordinates: ('
            f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * image_width}, '
            f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y * image_height})'
        )

        annotated_image = image.copy()
        # Draw segmentation on the image.
        # To improve segmentation around boundaries, consider applying a joint
        # bilateral filter to "results.segmentation_mask" with "image".
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR
        annotated_image = np.where(condition, annotated_image, bg_image)
        # Draw pose landmarks on the image.
        mp_drawing.draw_landmarks(
            annotated_image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)
        # Plot pose world landmarks.
        mp_drawing.plot_landmarks(
            results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)

